## Импорт библиотек

In [1]:
import requests
import psycopg2
import datetime

In [5]:
api_key = '61O5P07NNP5BYSU2'# API ключ(по факту влияет IP)
# Выбранные компании
companies = {
  'IBM': 'International Business Machines Corp',
  'YNDX': 'Yandex',
  'MIME': 'Mimecast Ltd'
}

id_key, limit = 0, 0  # Вспомогательные переменные
# Подключение к БД, которое в Docker на порту 5000
connection = psycopg2.connect(
  user="postgres", password="129053", host="localhost", port="5000", database="data_mart")
cursor = connection.cursor()
# Проход по всем символам выбранных компаний
for symbol in list(companies.keys()):
  # Запрос с параметрами
  url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&outputsize=full&apikey={api_key}"
  
  response = requests.get(url).json()  # Ответ в виде JSON
  data = response['Time Series (Daily)']  # Выбираем нужные данные
  # проход по этим интервалам
  for j in data.keys():
    # Запрос в БД
    query = f'INSERT INTO companies_data ("Date_point", "Name", "Open", "High", "Low", "Close", "Volume") VALUES (%s, %s, %s, %s, %s, %s, %s);'
    # Параметры получаем для конкретного промежутка 
    date_time = datetime.datetime.strptime(j, '%Y-%m-%d')
    name = companies[symbol]
    open = float(data[j]['1. open'])
    high = float(data[j]['2. high'])
    low = float(data[j]['3. low'])
    close = float(data[j]['4. close'])
    volume = float(data[j]['5. volume'])
    cursor.execute(query, (date_time, name, open, high, low, close, volume))  # Выполнение запроса
  connection.commit()  # Занос изменений в БД
      